In [ ]:
!pip install category-encoders

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import shap
import joblib

In [ ]:
raw_data = pd.read_csv('/ml/algorithmlabs.inference/inference_data.csv')
te = joblib.load('/ml/algorithmlabs.inference/pickle/encoder.pkl')
scaler = joblib.load('/ml/algorithmlabs.inference/pickle/scaler.pkl')
voting = joblib.load('/ml/algorithmlabs.inference/pickle/model.pkl')
explainer_1 = joblib.load('/ml/algorithmlabs.inference/pickle/shap_explainer_1.pkl')
explainer_2 = joblib.load('/ml/algorithmlabs.inference/pickle/shap_explainer_2.pkl')
explainer_3 = joblib.load('/ml/algorithmlabs.inference/pickle/shap_explainer_3.pkl')
explainer_4 = joblib.load('/ml/algorithmlabs.inference/pickle/shap_explainer_4.pkl')
name_dict = joblib.load('/ml/algorithmlabs.inference/pickle/name_dict.pkl')

In [ ]:
en_data = raw_data.copy()

In [ ]:
# 0208 수정
en_data.drop(['YYHAKGI', 'HAKJ_HAKBUN', 'HAKJ_JUMIN_NO', 'DAEHAK_NAME', 'HAKGWA_NAME', 'JUNGONG_NAME', 'BUJ_NAME', 'BS1_NAME', 'BS2_NAME', 'BS3_NAME', 'BD_GBN_NAME'], axis=1, inplace=True)

In [ ]:
object_cols = en_data.select_dtypes('object').columns
en_data[object_cols] = te.transform(en_data[object_cols])

In [ ]:
en_scaled = scaler.transform(en_data)
en_data = pd.DataFrame(data=en_scaled, index=en_data.index, columns=en_data.columns)

In [ ]:
pred_data = voting.predict_proba(en_data)[:,1]
raw_data['predict_proba'] = pred_data

In [ ]:
raw_data = raw_data[raw_data.columns[1:2].tolist()+raw_data.columns[-1:].tolist()+raw_data.columns[:1].tolist() + raw_data.columns[2:-1].tolist()]
raw_data.drop(['HAKJ_DAEHAK', 'HAKJ_HAKGWA', 'HAKJ_JUNGONG', 'HAKJ_BUJ_CODE', 'HAKJ_BS1_CODE', 'HAKJ_BS2_CODE', 'HAKJ_MACHUMJ_CODE'], axis=1, inplace=True) 

In [ ]:
for name in raw_data.columns.tolist() : 
    for name_ex in name_dict.keys() : 
        if name == name_ex : 
            raw_data.rename(columns={name : name_dict[name_ex]}, inplace=True)

In [ ]:
raw_data.rename(columns={'DAEHAK_NAME' : name_dict['HAKJ_DAEHAK']}, inplace=True)
raw_data.rename(columns={'HAKGWA_NAME' : name_dict['HAKJ_HAKGWA']}, inplace=True)
raw_data.rename(columns={'JUNGONG_NAME' : name_dict['HAKJ_JUNGONG']}, inplace=True)
raw_data.rename(columns={'BUJ_NAME' : name_dict['HAKJ_BUJ_CODE']}, inplace=True)
raw_data.rename(columns={'BS1_NAME' : name_dict['HAKJ_BS1_CODE']}, inplace=True)
raw_data.rename(columns={'BS2_NAME' : name_dict['HAKJ_BS2_CODE']}, inplace=True)
raw_data.rename(columns={'BS3_NAME' : name_dict['HAKJ_MACHUMJ_CODE']}, inplace=True)

In [ ]:
shap_values_1 = explainer_1.shap_values(en_data, check_additivity=False) 
shap_values_2 = explainer_2.shap_values(en_data, check_additivity=False) 
shap_values_3 = explainer_3.shap_values(en_data, check_additivity=False) 
shap_values_4 = explainer_4.shap_values(en_data, check_additivity=False)
fw = (shap_values_1[1]+shap_values_2+shap_values_3[1]+shap_values_4)/4
feature_weight = pd.DataFrame(fw, columns=en_data.columns)

In [ ]:
feature_weight['HAKJ_HAKBUN'] = raw_data['학번']
feature_weight = feature_weight[feature_weight.columns[-1:].tolist() + feature_weight.columns[:-1].tolist()]

In [ ]:
for name in feature_weight.columns.tolist() : 
    for name_ex in name_dict.keys() : 
        if name == name_ex : 
            feature_weight.rename(columns={name : name_dict[name_ex]}, inplace=True)

In [ ]:
raw_data.to_csv('/ml/algorithmlabs.inference/infer_result_dropout.csv', index=False)
feature_weight.to_csv('/ml/algorithmlabs.inference/feature_weight_dropout.csv', index=False)